<a href="https://colab.research.google.com/github/Jming9638/MachineLearning/blob/main/IdendityResolutionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install recordlinkage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import string
import recordlinkage

from recordlinkage.preprocessing import clean, phonetic
from recordlinkage.index import Full

In [ ]:
def process_text(text):
    text = str(text)
    lower_text = text.lower()
    no_punctuation_text = lower_text.translate(str.maketrans('', '', string.punctuation))
    final_text = no_punctuation_text.replace(' ', '')
    return final_text

In [ ]:
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file('./bq-service-account.json')

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(credentials=credentials)

query = """
SELECT *
FROM `spheric-terrain-312804.identity_resolution_us.active_us`
"""
df_active = client.query(query).to_dataframe()

query = """
SELECT *
FROM `spheric-terrain-312804.identity_resolution_us.shopify_us`
"""
df_shopify = client.query(query).to_dataframe()

In [ ]:
data = pd.concat([df_active, df_shopify], ignore_index=True)
data = data.drop(['gender', 'age'], axis=1)
data.head()

,uuid,first_name,last_name,phone,email,address,zipcode,post_Town,state,country,region,continent
0,active_0b7b5536,Rob,Lee,+16411050868,rna1920@outlook.com,1268 Vallejo Center,47239,Bloomingdale,Kansas,Nauru,Iowa,Asia
1,active_901edfb3,Cris,Farmer,+13085813884,engines1936@protonmail.com,482 Thor Junction,30644,Lawndale,Tennessee,Dominica,Iowa,Asia
2,active_367942c8,Kena,Pitts,+1-313-369-1737,locally1869@duck.com,241 Forest Knolls Run,76761,Cuyahoga Falls,Utah,Guatemala,Iowa,Asia
3,active_05a04ef9,Nick,Schmidt,+1-240-538-2219,reasonable1977@duck.com,543 Crissy Field Plaza,39556,Plant City,South Dakota,Norway,Iowa,Asia
4,active_6df8da1a,Otha,Page,+1-325-108-4701,emily1888@yahoo.com,190 Skyview Lake,31847,Lowell,Louisiana,Jersey,Iowa,Asia


In [ ]:
for col in data.columns:
    if col != 'uuid':
        if col != 'email':
            data[col] = data[col].apply(process_text)
        else:
            data[col] = data[col].apply(lambda x: str(x).split('@')[0])
            data[col] = data[col].apply(process_text)

In [ ]:
def fast_recordlinkage(data, required_cols):
    data = data.set_index('uuid')
    data1 = data.copy()
    data2 = data.copy()

    print('Total iterations:', data1.shape[0] * data2.shape[0] * len(required_cols))

    indexer = recordlinkage.Index()
    indexer.add(Full())
    candidate_links = indexer.index(data1, data1)

    compare = recordlinkage.Compare(n_jobs=-1)
    for feature in required_cols:
        if feature not in ['phone', 'zipcode']:
            compare.string(feature, feature, method='jarowinkler', label=feature)
        else:
            compare.string(feature, feature, method='levenshtein', label=feature)

    matches = compare.compute(candidate_links, data1, data2)
    matches.columns = [x + '_score' for x in matches.columns]
    matches = matches.reset_index()
    data1 = data1.reset_index()
    data2 = data2.reset_index()
    data1.columns = [c + '_1' for c in data1.columns]
    data2.columns = [c + '_2' for c in data2.columns]

    matches = matches.merge(data1, on='uuid_1', how='left')
    matches = matches.merge(data2, on='uuid_2', how='left')
    matches = matches.set_index(['uuid_1', 'uuid_2'])
    matches = matches[sorted(matches.columns)]
    matches['matching_rate'] = matches[[s for s in matches.columns if '_score' in s]].mean(axis=1)

    return matches

In [ ]:
required_cols = [col for col in data.columns if col != 'uuid']
matches = fast_recordlinkage(data.sample(frac=0.1, random_state=42), required_cols)

Total iterations: 29802476


In [ ]:
matches

address_1           address_2  \
uuid_1          uuid_2                                                     
active_574e71d3 active_574e71d3         266fargomews        266fargomews   
                active_c683f603         266fargomews      1226emerytrail   
                shopify_8a754252        266fargomews      921lestercreek   
                shopify_f90999a9        266fargomews     1310hofflanding   
                active_9a242cf8         266fargomews  677mccoppinstation   
...                                              ...                 ...   
active_d0d7fa34 active_582c539c   695armisteadstreet    1215mayfaircourt   
                active_69bc7e23   695armisteadstreet    1357robbleefield   
                active_b36e7814   695armisteadstreet  430santamonicapark   
                shopify_9882a192  695armisteadstreet  1125acevedoviaduct   
                active_d0d7fa34   695armisteadstreet  695armisteadstreet   

                                  address_score continent_1 continent_2  \
uuid_1          uuid_2                                                    
active_574e71d3 active_574e71d3        1.000000        asia        asia   
                active_c683f603        0.524603        asia   australia   
                shopify_8a754252       0.456349        asia  antarctica   
                shopify_f90999a9       0.372222        asia  antarctica   
                active_9a242cf8        0.435185        asia  antarctica   
...                                         ...         ...         ...   
active_d0d7fa34 active_582c539c        0.523148        asia   australia   
                active_69bc7e23        0.561177        asia  antarctica   
                active_b36e7814        0.449735        asia   australia   
                shopify_9882a192       0.546296        asia      africa   
                active_d0d7fa34        1.000000        asia        asia   

                                  continent_score   country_1  \
uuid_1          uuid_2                                          
active_574e71d3 active_574e71d3          1.000000        chad   
                active_c683f603          0.694444        chad   
                shopify_8a754252         0.566667        chad   
                shopify_f90999a9         0.566667        chad   
                active_9a242cf8          0.566667        chad   
...                                           ...         ...   
active_d0d7fa34 active_582c539c          0.694444  puertorico   
                active_69bc7e23          0.566667  puertorico   
                active_b36e7814          0.694444  puertorico   
                shopify_9882a192         0.775000  puertorico   
                active_d0d7fa34          1.000000  puertorico   

                                               country_2  country_score  \
uuid_1          uuid_2                                                    
active_574e71d3 active_574e71d3                     chad       1.000000   
                active_c683f603                    japan       0.483333   
                shopify_8a754252            ålandislands       0.555556   
                shopify_f90999a9               lithuania       0.574074   
                active_9a242cf8                   monaco       0.472222   
...                                                  ...            ...   
active_d0d7fa34 active_582c539c                 ethiopia       0.550000   
                active_69bc7e23                  stlucia       0.490476   
                active_b36e7814   palestinianterritories       0.625108   
                shopify_9882a192                 bahrain       0.495238   
                active_d0d7fa34               puertorico       1.000000   

                                        email_1  ...  region_1       region_2  \
uuid_1          uuid_2                           ...                            
active_574e71d3 active_574e71d3   miniature1991  ...  michigan       michigan   
             

In [ ]:
matches = matches.reset_index()
filters = (matches['matching_rate'] > 0.4)
deduped_data = matches[filters].reset_index(drop=True)
deduped_data


def aggregate_to_list(series):
    return list(series)


grouped_id = deduped_data.groupby(['uuid_1']).agg({'uuid_2': aggregate_to_list}).reset_index()


def merge_columns(row):
    return sorted(list(set([row['uuid_1']] + row['uuid_2'])))


grouped_id['merged'] = grouped_id.apply(merge_columns, axis=1)
grouped_id.drop_duplicates(subset=['merged'])

,uuid_1,uuid_2,merged
0,active_008b0a62,"[active_574e71d3, shopify_f90999a9, active_9a2...","[active_008b0a62, active_00f57040, active_0111..."
1,active_00f57040,"[active_c683f603, shopify_8a754252, active_9a2...","[active_008b0a62, active_00f57040, active_01c9..."
2,active_011129ce,"[active_c683f603, active_9a242cf8, shopify_cd4...","[active_008b0a62, active_011129ce, active_0148..."
3,active_0148eeba,"[shopify_8a754252, active_9a242cf8, active_f4a...","[active_008b0a62, active_011129ce, active_0148..."
4,active_01c9ad28,"[active_c683f603, shopify_f90999a9, active_9a2...","[active_00f57040, active_011129ce, active_01c9..."
...,...,...,...
1641,shopify_fe7fab58,"[active_574e71d3, active_c683f603, shopify_8a7...","[active_008b0a62, active_00f57040, active_0111..."
1642,shopify_feef5707,"[active_574e71d3, active_c683f603, shopify_cd4...","[active_008b0a62, active_00f57040, active_0111..."
1643,shopify_ffa7bf61,"[active_c683f603, shopify_8a754252, shopify_f9...","[active_00f57040, active_0148eeba, active_01c9..."
1644,shopify_ffd5d509,"[active_574e71d3, shopify_8a754252, shopify_f9...","[active_008b0a62, active_011129ce, active_0148..."
